# Предсказание стоимости жилья

В проекте обучим модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных предскажем медианную стоимость дома в жилом массиве. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели используем метрики `RMSE`, `MAE` и `R2`.

В колонках датасета содержатся следующие данные:
- `longitude` — широта;
- `latitude` — долгота;
- `housing_median_age` — медианный возраст жителей жилого массива;
- `total_rooms` — общее количество комнат в домах жилого массива;
- `total_bedrooms` — общее количество спален в домах жилого массива;
- `population` — количество человек, которые проживают в жилом массиве;
- `households` — количество домовладений в жилом массиве;
- `median_income` — медианный доход жителей жилого массива;
- `median_house_value` — медианная стоимость дома в жилом массиве;
- `ocean_proximity` — близость к океану.

**План по проекта**

1. Инициализация Spark-сессии: Создать локальную Spark-сессию.
2. Чтение и загрузка данных: Прочитать данные из файла `###.csv`.
3. Исследование данных: Вывести типы данных колонок датасета, исследовать наличие пропусков и выполнить их заполнение, при необходимости преобразовать категориальные значения техникой `One hot encoding`.
4. Построение модели: Подготовить данные и построить две модели линейной регрессии — на всех данных из файла и только на числовых переменных (исключив категориальные).
5. Оценка модели: Сравнить результаты работы обеих моделей по метрикам `RMSE`, `MAE` и `R2`.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1">Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1">Вывод</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2">Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1">Вывод</a></span></li></ul></li><li><span><a href="#Анализ-результатов" data-toc-modified-id="Анализ-результатов-3">Анализ результатов</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1">Вывод</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4">Общий вывод</a></span></li></ul></div>

## Подготовка данных

In [1]:
#импорт библиотек и модулей
import pandas as pd 
import numpy as np
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, count, when, sum

RANDOM_SEED = 42

Инициализируем локальную Spark-сессию:

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("california_housing") \
                    .getOrCreate()

In [3]:
# чтение содержимого файла
df = spark.read.csv("###.csv", inferSchema=True, header=True)

In [4]:
# названия колонок 
print(pd.DataFrame(df.dtypes, columns=['column', 'type']))

# первые 10 строк 
df.show(10)

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500

Колонка `ocean_proximity` хранит категориальные значения, в остальных колонках хранятся количественные данные.

In [5]:
# количество строк
print("Количество строк: ", df.count())

Количество строк:  20640


In [6]:
# базовые статистики
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [7]:
# проверка пропусков
missing_values = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).toPandas()

missing_values = missing_values.T.reset_index()
missing_values.columns = ['Столбец', 'Количество пропущенных значений']
print(missing_values)

              Столбец  Количество пропущенных значений
0           longitude                                0
1            latitude                                0
2  housing_median_age                                0
3         total_rooms                                0
4      total_bedrooms                              207
5          population                                0
6          households                                0
7       median_income                                0
8  median_house_value                                0
9     ocean_proximity                                0


In [8]:
# Преобразуем DataFrame PySpark в Pandas DataFrame
df_pandas = df.toPandas()

print('Корреляция c total_bedrooms (Спирмена):')
for c in df_pandas.columns:
    # Проверяем, является ли столбец числовым
    if c != 'total_bedrooms' and pd.api.types.is_numeric_dtype(df_pandas[c]):
        correlation = df_pandas['total_bedrooms'].corr(df_pandas[c], method='spearman')
        print(f"  {c:<25} {correlation:>5.2f}")

Корреляция c total_bedrooms (Спирмена):
  longitude                  0.06
  latitude                  -0.06
  housing_median_age        -0.31
  total_rooms                0.92
  population                 0.87
  households                 0.98
  median_income             -0.01
  median_house_value         0.09


В колонке `total_bedrooms` есть пропущенные значения. Наибольшая корреляция у этого признака с `households`. Заполним `null` значения в `total_bedrooms` через медиану соотношения `households` к `total_bedrooms`.

In [9]:
# Рассчитываем коэффициенты соотношения households к total_bedrooms
df_with_ratio = df.withColumn("households_to_bedrooms", col("households") / col("total_bedrooms"))

# Рассчитываем медиану
median_value = df_with_ratio.approxQuantile("households_to_bedrooms", [0.5], 0.01)[0]

# Заполняем пропуски в total_bedrooms
df = df_with_ratio.withColumn(
    "total_bedrooms",
    when(col("total_bedrooms").isNull(), col("households") / median_value)
    .otherwise(col("total_bedrooms"))
)

# Удаляем временный столбец households_to_bedrooms
df = df.drop("households_to_bedrooms")

In [10]:
# проверка пропусков
missing_values = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).toPandas()

missing_values = missing_values.T.reset_index()
missing_values.columns = ['Столбец', 'Количество пропущенных значений']
print(missing_values)

              Столбец  Количество пропущенных значений
0           longitude                                0
1            latitude                                0
2  housing_median_age                                0
3         total_rooms                                0
4      total_bedrooms                                0
5          population                                0
6          households                                0
7       median_income                                0
8  median_house_value                                0
9     ocean_proximity                                0


In [11]:
# проверка категориальных значений
df.select('ocean_proximity').distinct().collect()

[Row(ocean_proximity='ISLAND'),
 Row(ocean_proximity='NEAR OCEAN'),
 Row(ocean_proximity='NEAR BAY'),
 Row(ocean_proximity='<1H OCEAN'),
 Row(ocean_proximity='INLAND')]

Неявных дубликатов, ошибок не наблюдается.

### Вывод

- Колонка ocean_proximity хранит категориальные значения, в остальных колонках хранятся количественные данные.
- Пропуски в `total_bedrooms` заполнены значениями через медиану соотношения `households` к `total_bedrooms`.
- Неявных дубликатов, ошибок не наблюдается.

## Обучение моделей

Разделим признаки на числовые и категориальные, выделим целевой признак.

In [12]:
cat_features = ['ocean_proximity']
num_features = ['longitude',
                'latitude',
                'housing_median_age',
                'total_rooms',
                'total_bedrooms',
                'population',
                'households',
                'median_income'
               ]
target = 'median_house_value'

Разделим датасет на обучающую и тестовую выборки.

In [13]:
train_data, test_data = df.randomSplit([.8, .2], seed=RANDOM_SEED)

print(f'''
    Обучающая выборка {train_data.count()} 
    Тестовая выборка {test_data.count()}
    ''')


    Обучающая выборка 16560 
    Тестовая выборка 4080
    


Масштабируем и кодируем признаки:

In [14]:
# Кодирование категориальных признаков на обучающей выборке
indexer = StringIndexer(inputCols=cat_features, 
                        outputCols=[c+'_idx' for c in cat_features]) 
indexer_model = indexer.fit(train_data)  # Обучаем индексатор на обучающих данных
train_data = indexer_model.transform(train_data)

encoder = OneHotEncoder(inputCols=[c+'_idx' for c in cat_features],
                        outputCols=[c+'_ohe' for c in cat_features])
encoder_model = encoder.fit(train_data)  # Обучаем энкодер на обучающих данных
train_data = encoder_model.transform(train_data)

# Создание векторов для категориальных признаков
categorical_assembler = VectorAssembler(inputCols=[c+'_ohe' for c in cat_features],
                                        outputCol="cat_features")
train_data = categorical_assembler.transform(train_data)

# Создание векторов для числовых признаков
numerical_assembler = VectorAssembler(inputCols=num_features,
                                      outputCol="num_features")
train_data = numerical_assembler.transform(train_data)

# Стандартизация числовых признаков
standardScaler = StandardScaler(inputCol='num_features',
                                outputCol="num_features_scaled")
scaler_model = standardScaler.fit(train_data)  # Обучаем стандартный скейлер на обучающих данных
train_data = scaler_model.transform(train_data)

# Объединение всех признаков
all_features = ['cat_features', 'num_features_scaled']
final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train_data = final_assembler.transform(train_data)

# Применение тех же трансформеров к тестовым данным
test_data = indexer_model.transform(test_data)  # Применяем обученную модель индексатора
test_data = encoder_model.transform(test_data)  # Применяем обученную модель энкодера
test_data = categorical_assembler.transform(test_data)
test_data = numerical_assembler.transform(test_data)
test_data = scaler_model.transform(test_data)  # Применяем обученную модель скейлера
test_data = final_assembler.transform(test_data)

# Проверка результатов
train_data.select(all_features).show(3)
test_data.select(all_features).show(3)

+-------------+--------------------+
| cat_features| num_features_scaled|
+-------------+--------------------+
|(4,[2],[1.0])|[-61.931952286653...|
|(4,[2],[1.0])|[-61.907050013920...|
|(4,[2],[1.0])|[-61.892108650280...|
+-------------+--------------------+
only showing top 3 rows

+-------------+--------------------+
| cat_features| num_features_scaled|
+-------------+--------------------+
|(4,[2],[1.0])|[-61.907050013920...|
|(4,[2],[1.0])|[-61.872186832094...|
|(4,[2],[1.0])|[-61.872186832094...|
+-------------+--------------------+
only showing top 3 rows



Построим модели линейной регрессии, первую на всех данных и вторую только на числовых переменных (исключив категориальные).

In [15]:
lr_1 = LinearRegression(labelCol=target, featuresCol='features',regParam=0.0)
model_1 = lr_1.fit(train_data) 

25/01/16 13:12:58 WARN Instrumentation: [a684d9bb] regParam is zero, which might cause numerical instability and overfitting.
25/01/16 13:12:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
25/01/16 13:12:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
25/01/16 13:12:59 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
25/01/16 13:12:59 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [16]:
lr_2 = LinearRegression(labelCol=target, featuresCol='num_features_scaled', regParam=0.0)
model_2 = lr_2.fit(train_data) 

25/01/16 13:13:01 WARN Instrumentation: [c5cce1c0] regParam is zero, which might cause numerical instability and overfitting.


### Вывод

Проведено разделение данных на обучающие и тестовые выборки, обучение модели линейной регрессии.

## Анализ результатов

Предсказание вероятности моделей.

In [17]:
predictions_1 = model_1.transform(test_data)

In [18]:
predictions_2 = model_2.transform(test_data)

In [19]:
# Оценка качества модели на всех данных
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_1)
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_1)
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_1)

print("Метрики модели 1:")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R2: {r2}")

Метрики модели 1:
RMSE: 70600.5966926188
MAE: 50771.63521003129
R2: 0.6397445918050668


In [20]:
# Оценка качества модели только на числовых переменных
rmse_num = evaluator.evaluate(predictions_2, {evaluator.metricName: "rmse"})
mae_num = evaluator.evaluate(predictions_2, {evaluator.metricName: "mae"})
r2_num = evaluator.evaluate(predictions_2, {evaluator.metricName: "r2"})

print("Метрики модели 2:")
print(f"RMSE: {rmse_num}")
print(f"MAE: {mae_num}")
print(f"R2: {r2_num}")

Метрики модели 2:
RMSE: 71561.22911187526
MAE: 51667.456968399754
R2: 0.6298742089701994


In [21]:
# Завершение Spark сессии
spark.stop()

### Вывод

Анализ показывает, модель 1 более точно предсказывает целевую переменную по сравнению с моделью 2. Меньшая `RMSE` свидетельствует о меньшем среднем квадрате ошибок между предсказанными и фактическими значениями.
Модель 1 имеет более низкое значение `MAE`, что также свидетельствует о ее большей точности в предсказаниях. Модель 1 более последовательна в своих предсказаниях, поскольку среднее абсолютное отклонение от фактических значений меньше, чем в модели 2. Оба значения `R2` меньше 1, что указывает на то, что ни одна из моделей не объясняет всю вариацию в данных, но модель 1 делает это лучше.


## Общий вывод

В ходе проекта была выполнена работа по обучению модели линейной регрессии для предсказания медианной стоимости жилья в Калифорнии на основе данных 1990 года. Были выполнены следующие этапы:

1. Инициализация Spark-сессии и чтение данных из файла.
2. Предобработка данных, включая обработку пропусков и преобразование категориальных переменных с помощью `One Hot Encoding`.
3. Масштабирование числовых признаков и разделение данных на обучающую и тестовую выборки.
4. Построение двух моделей линейной регрессии: одна с использованием всех данных, вторая — только на числовых переменных.
4. Оценка качества моделей с помощью метрик `RMSE`, `MAE` и `R2`.

Результаты:

Модель, обученная на всех данных, показала лучшие результаты по всем метрикам по сравнению с моделью, обученной только на числовых переменных:
- `RMSE`: 70600.5966926188
- `MAE`: 50771.63521003129
- `R2`: 0.6397445918050668

После завершения работы с моделью Spark-сессия была корректно закрыта.